In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from queue import Queue

from IPython.core.debugger import set_trace

## Part I

In [16]:
test_schematic = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L'''

In [65]:
with open('input.txt') as inp:
    in_schematic = inp.read()

In [ ]:
# tree data structure with stars as nodes, direct orbits as children

In [76]:
class star:
    def __init__(self, name = None):
        self.direct_orbits = []
        
        self.n_direct_orbits = 0
        self.n_indirect_orbits = 0
        self.name = name
        self.updated = False
        
    def add_orbiter(self, other):
        self.direct_orbits.append(other)
        self.n_direct_orbits += 1

    def update_indirect_orbits(self):
        if self.updated:
            return self.n_indirect_orbits
        else:
            self.updated = True
            
        if not self.direct_orbits:
            self.n_indirect_orbits = 0
        else:
            for orbiter in self.direct_orbits:
                self.n_indirect_orbits += orbiter.n_direct_orbits + orbiter.update_indirect_orbits()
        return self.n_indirect_orbits        
    
    def __repr__(self):
        return '{}'.format([n.name for n in self.direct_orbits])
        

In [67]:
def star_map_from_schematic(schematic):
    relationships = schematic.split('\n')
    star_map = {}
    
    for orbit in relationships:
        stars = orbit.split(')')
        orbiter = stars[1]
        orbited = stars[0]

        if orbiter not in star_map:
            star_map[orbiter] = star(orbiter)
        
        if orbited in star_map:
            star_map[orbited].add_orbiter( star_map[orbiter] )
        else:
            star_map[orbited] = star(orbited)
            star_map[orbited].add_orbiter( star_map[orbiter] )
            
    return star_map

In [71]:
star_map = star_map_from_schematic(in_schematic)

In [72]:
q = Queue()
q.put(star_map['COM'])
total_orbits = 0
while not q.empty():
    node = q.get_nowait()
    node.update_indirect_orbits()
    total_orbits += node.n_direct_orbits + node.n_indirect_orbits
    for n in node.direct_orbits:
        q.put(n)

In [144]:
total_orbits

241064

## Part II

In [118]:
test_schematic = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN'''

In [ ]:
# graph data structure, copied method

In [122]:
class Graph(object):
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict


    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.__graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        edge = set(edge)
        vertex1 = edge.pop()
        if edge:
            # not a loop
            vertex2 = edge.pop()
        else:
            # a loop
            vertex2 = vertex1
        if vertex1 in self.__graph_dict:
            self.__graph_dict[vertex1].append(vertex2)
        else:
            self.__graph_dict[vertex1] = [vertex2]

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            for neighbour in self.__graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges

    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def find_isolated_vertices(self):
        """ returns a list of isolated vertices. """
        graph = self.__graph_dict
        isolated = []
        for vertex in graph:
            print(isolated, vertex)
            if not graph[vertex]:
                isolated += [vertex]
        return isolated

    def find_path(self, start_vertex, end_vertex, path=[]):
        """ find a path from start_vertex to end_vertex 
            in graph """
        graph = self.__graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return path
        if start_vertex not in graph:
            return None
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_path = self.find_path(vertex, 
                                               end_vertex, 
                                               path)
                if extended_path: 
                    return extended_path
        return None
    

    def find_all_paths(self, start_vertex, end_vertex, path=[]):
        """ find all paths from start_vertex to 
            end_vertex in graph """
        graph = self.__graph_dict 
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return [path]
        if start_vertex not in graph:
            return []
        paths = []
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_paths = self.find_all_paths(vertex, 
                                                     end_vertex, 
                                                     path)
                for p in extended_paths: 
                    paths.append(p)
        return paths

In [145]:
def graph_from_schematic(schematic):
    relationships = schematic.split('\n')
    graph = {}
    
    for orbit in relationships:
        stars = orbit.split(')')
        orbiter = stars[1]
        orbited = stars[0]

        if orbiter not in graph:
            graph[orbiter] = []
            
        if orbited not in graph:
            graph[orbited] = []
        
        graph[orbiter].append( orbited )
        graph[orbited].append( orbiter )
            
    return Graph(graph)

In [156]:
g = graph_from_schematic(in_schematic)

In [157]:
# due to the tree structure of the graph, this path is guaranteed to be the shortest
path = g.find_path('YOU', 'SAN')
print(path)

['YOU', '18W', 'H5C', '1LV', '6QP', 'TN5', '48B', '11M', 'S2Q', '7D5', 'HMG', 'P81', '6RV', 'TBN', '5RG', 'VVR', '22V', 'Q95', 'D68', 'QS4', 'YD9', '3G9', 'D73', 'HK9', 'SVT', '2SM', 'YSB', '463', 'FFW', 'KMC', '516', '51B', '2YC', 'F2Z', 'RCR', '75B', 'RLM', 'B89', 'S49', '2WT', 'ZY4', 'JLS', 'BCC', 'CTB', 'HGK', '7DD', 'VVQ', '7FL', 'J8T', '424', 'FXX', 'ZWV', '31M', 'J8N', 'XQ1', 'JMB', '6M7', 'H6N', 'D4M', 'TRY', 'DM2', 'LBR', '79V', 'F6S', 'S2N', 'MBD', 'VNR', 'YD3', 'JTD', 'CB6', 'WP5', '8Z4', 'SGK', 'LCL', 'NTV', 'NKW', '2NH', '49H', '7WJ', 'Q89', 'V6Y', 'QK9', 'R3H', '9BQ', 'ZFF', 'Z7R', '6YN', 'ZD1', 'CS7', 'WQM', 'C91', '1NZ', 'ZY1', 'LSY', 'LKG', '6DD', '523', 'XCN', 'ZFD', 'N1G', 'QD8', 'VK3', 'FF2', '2PH', 'GZZ', 'YG6', 'C2Q', 'MK7', 'J71', 'H1F', '17T', 'LZW', 'PHX', 'XJJ', 'MHC', 'NZ1', 'V6P', '7H1', 'C1C', 'SL9', 'ZTH', 'NC6', 'B6R', 'FK6', '7RG', '7DZ', '2YN', 'B1W', 'T6W', 'D2K', 'R54', 'J53', 'KSS', 'LNV', '9QQ', 'JDB', 'SRF', '3CS', 'RDK', '79T', 'H56', 'GYS', 'YZG'

In [160]:
# the path includes YOU and SAN, so we remove those. In addition, we only need to orbit the same planet 
# santa, not orbit santa, so we remove an additional step (total 3)
n_jumps = len(path)-3
n_jumps

418